In [4]:
import subprocess
import os
import numpy as np
import re
import itertools
import pandas as pd
from Bio import SeqIO
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

### Gathering the genomes with their names associated

Dict : GCF -> (Specie name,fnaPath,faaPath)

In [ ]:
def DownloadingSequences(bacteriaPath,archeaPath,outputPath):
    
    print("Downloading Bacteria Files")
    subprocess.run("ncbi-genome-download --genera \'"+bacteriaPath+"\' bacteria -F protein-fasta,fasta -o "+outputPath,shell=True)
    
    print("Downloading Archaea Files")
    subprocess.run("ncbi-genome-download --genera \'"+archeaPath+"\' archaea -F protein-fasta,fasta -o "+outputPath,shell=True)
        
    CleaningFolder(outputPath)	

def CleaningFolder(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.gz'):
                subprocess.run("gunzip "+root+"/"+file,shell=True)
            if file=="MD5SUMS":
                subprocess.run("rm "+root+"/"+file,shell=True)
            
            
DownloadingSequences('../../data/Bacteria.list','../../data/Archea.list','../../data/')

In [5]:
#Returns a dictionnary: Key: Phylum -> Value Other dictionnary | Organism name -> Path to Proteome and Genome

def ParsingSequences(path):
    #Dictionnary of all the adresses
    dictGen={}
    #Getting the species names via Regular expression
    regex="[0-9] (.+)(chromosome|,)"
    
    listPhylums=os.listdir(path)
    listFiles=[[] for _ in range(len(listPhylums))]
    for i in range(len(listPhylums)):
        listfaa=[]
        listfna=[]
        listNames=[]
        for root,dirs,files in os.walk(path+'/'+listPhylums[i]):
            for file in files:
                if file.endswith('.faa'):
                    listfaa.append(root+'/'+file)
                elif file.endswith('.fna'):
                    listfna.append(root+'/'+file)
                    lineSpecie=open(root+'/'+file).readlines()[0]
                    match=re.search(regex,lineSpecie).group(1)
                    if match.split(' ')[-1]=='chromosome':
                        match=' '.join(match.split(' ')[:-1])
                    listNames.append(match)    
        dictGen[listPhylums[i]]=dict(zip(listNames,zip(listfaa,listfna)))          
        
    return dictGen

In [6]:
pathGenomes='../../data/refseq/'
dictGeneral=ParsingSequences(pathGenomes)

print("Number of Bacterias:",len(dictGeneral['bacteria'].keys()))
print("Number of Archaeas:",len(dictGeneral['archaea'].keys()))

Number of Bacterias: 99
Number of Archaeas: 44


### Missing organisms

In [7]:
listBacterias=[i[:-1] for i in open('../../data/Bacteria.list','r').readlines()[1:]]
listArchaeas=[i[:-1] for i in open('../../data/Archea.list','r').readlines()[1:]]


print('Bacterias:','\n')
for i in listBacterias:
    if i not in dictGeneral['bacteria'].keys():
        print(i)
        
print('Archaeas:','\n')    
for i in listArchaeas:
    if i not in dictGeneral['archaea'].keys():
        print(i)

Bacterias: 

Borrelia burgdorferi B31
Candidatus Cloacamonas acidaminovorans
Candidatus Endomicrobium sp. Rs-D17
Cupriavidus taiwanensis
Cupriavidus taiwanensis
Cyanothece sp. ATCC 51142
Cyanothece sp. ATCC 51142
Dehalococcoides ethenogenes 195
Deinococcus radiodurans R1
Deinococcus radiodurans R1
Fusobacterium nucleatum subsp. nucleatum ATCC 25586
Gemmata obscuriglobus UQM 2246
Leptospira interrogans serovar Lai str. 56601
Leptospira interrogans serovar Lai str. 56601
Magnetococcus sp. MC-1
Solibacter usitatus Ellin6076
Thermoanaerobacter tengcongensis MB4
Thermobaculum terrenum ATCC BAA-798
Thermobaculum terrenum ATCC BAA-798
Thermus thermophilus HB8
Archaeas: 

Candidatus Methanoregula boonei 6A8
Candidatus Methanosphaerula palustris E1-9c
Cenarchaeum symbiosum A
Desulfurococcus kamchatkensis 1221n
Halobacterium sp. NRC-1
Halorubrum lacusprofundi ATCC 49239
Halorubrum lacusprofundi ATCC 49239
Methanosaeta thermophila PT
Nanoarchaeum equitans Kin4-M
Sulfolobus solfataricus P2
Thermop

### Read sequence from file

In [8]:
def ReadSequence(path):
    seqs=SeqIO.parse(path,'fasta')
    seqs=[str(seq.seq) for seq in seqs]
    return ''.join(seqs)

### Compute K-Mers

In [35]:
def Cut(sequence,size):
    return [sequence[i:i+size] for i in range(len(sequence)-(size-1)) ]

def CountCuts(listOfSequences,normalized=False):
    
    #Creating the dictionnary
    possibilities=list(map(''.join,list(itertools.product('ACGT', repeat=len(listOfSequences[0])))))
    counts=[0 for i in range(len(possibilities))]
    dicoCuts=dict(zip(possibilities,counts))
    
    #Counting sequences
    for sequence in listOfSequences:
        try:
            dicoCuts[sequence]+=1
        except:
            None
    
    #Conversion to df
    df=pd.DataFrame([dicoCuts])

    
    if normalized==False:        
        return df
    else:
        return df/np.sum(df.values)
 
def CountKMerFromFile(path,size):
    sequence=ReadSequence(path)
    cutList=Cut(sequence,size)
    return CountCuts(cutList,normalized=True)

### Quick Look at the dataset

In [10]:
pathTest=dictGeneral['archaea']['Methanosarcina mazei Go1']
print(pathTest[0])

../../data/refseq//archaea/GCF_000007065.1/GCF_000007065.1_ASM706v1_protein.faa


In [11]:
seqTest=ReadSequence(pathTest[1])
seqCuttesTest=Cut(seqTest,3)
dicSeq=CountCuts(seqCuttesTest,normalized=True)
dicSeq

['AAA', 'AAC', 'AAG', 'AAT', 'AAN', 'AAR', 'ACA', 'ACC', 'ACG', 'ACT', 'ACN', 'ACR', 'AGA', 'AGC', 'AGG', 'AGT', 'AGN', 'AGR', 'ATA', 'ATC', 'ATG', 'ATT', 'ATN', 'ATR', 'ANA', 'ANC', 'ANG', 'ANT', 'ANN', 'ANR', 'ARA', 'ARC', 'ARG', 'ART', 'ARN', 'ARR', 'CAA', 'CAC', 'CAG', 'CAT', 'CAN', 'CAR', 'CCA', 'CCC', 'CCG', 'CCT', 'CCN', 'CCR', 'CGA', 'CGC', 'CGG', 'CGT', 'CGN', 'CGR', 'CTA', 'CTC', 'CTG', 'CTT', 'CTN', 'CTR', 'CNA', 'CNC', 'CNG', 'CNT', 'CNN', 'CNR', 'CRA', 'CRC', 'CRG', 'CRT', 'CRN', 'CRR', 'GAA', 'GAC', 'GAG', 'GAT', 'GAN', 'GAR', 'GCA', 'GCC', 'GCG', 'GCT', 'GCN', 'GCR', 'GGA', 'GGC', 'GGG', 'GGT', 'GGN', 'GGR', 'GTA', 'GTC', 'GTG', 'GTT', 'GTN', 'GTR', 'GNA', 'GNC', 'GNG', 'GNT', 'GNN', 'GNR', 'GRA', 'GRC', 'GRG', 'GRT', 'GRN', 'GRR', 'TAA', 'TAC', 'TAG', 'TAT', 'TAN', 'TAR', 'TCA', 'TCC', 'TCG', 'TCT', 'TCN', 'TCR', 'TGA', 'TGC', 'TGG', 'TGT', 'TGN', 'TGR', 'TTA', 'TTC', 'TTG', 'TTT', 'TTN', 'TTR', 'TNA', 'TNC', 'TNG', 'TNT', 'TNN', 'TNR', 'TRA', 'TRC', 'TRG', 'TRT', 'TRN'

AAA       AAC       AAG       AAT  AAN  AAR     ACA       ACC  \
0  0.040144  0.015372  0.022672  0.025282  0.0  0.0  0.0142  0.011276   

        ACG       ACT  ...  RNG  RNT  RNN  RNR  RRA  RRC  RRG  RRT  RRN  RRR  
0  0.006642  0.012942  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 216 columns]

In [12]:
dicSeq.values

array([[0.04014434, 0.015372  , 0.02267169, 0.02528182, 0.        ,
        0.        , 0.01419998, 0.01127591, 0.00664202, 0.01294179,
        0.        , 0.        , 0.0204895 , 0.01394854, 0.01737818,
        0.01294154, 0.        , 0.        , 0.02270562, 0.01644442,
        0.01557706, 0.02518149, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.01677374, 0.00819145, 0.01987651, 0.01555046,
        0.        , 0.        , 0.01216695, 0.01156153, 0.01026379,
        0.01735353, 0.        , 0.        , 0.0084429 , 0.00553347,
        0.01019104, 0.00661834, 0.        , 0.        , 0.00798542,
        0.01431838, 0.02013772, 0.02229476, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.02603224, 0.00

In [ ]:
plt.bar(np.arange(0,len(dicSeq.columns),1),dicSeq.values[0])

#### Computing some distances

In [38]:
distanceDf=pd.DataFrame(data=None,index=dictGeneral['bacteria'].keys(),columns=dictGeneral['bacteria'].keys())

In [47]:
keys=list(dictGeneral['bacteria'].keys())
for i in range(len(keys)):
    for j in range(i,len(keys)):
        if i!=j:
            print("Computing. ",(i*len(keys)+j)/50,'% completed')
            contingencyI=CountKMerFromFile(dictGeneral['bacteria'][keys[i]][1],3).values[0]
            contingencyJ=CountKMerFromFile(dictGeneral['bacteria'][keys[j]][1],3).values[0]
            contingency=np.vstack((contingencyI,contingencyJ))
            pVal=chi2_contingency(contingency)[0]
            print('Pvalue:',pVal)
            distanceDf.loc[keys[i],keys[j]]=pVal
            

Computing.  0.02 % completed
Pvalue: 4.590284871824413e-06
Computing.  0.04 % completed
Pvalue: 0.2988834119905005
Computing.  0.06 % completed
Pvalue: 0.00043166572874940947
Computing.  0.08 % completed
Pvalue: 0.34001631932231824
Computing.  0.1 % completed


KeyboardInterrupt: 

In [24]:
contingencyI.values

array([[4.87135197e-03, 8.78990425e-03, 1.05567767e-02, 4.81633701e-03,
        7.43445450e-07, 0.00000000e+00, 8.11158462e-03, 1.70840791e-02,
        1.66881200e-02, 5.74787415e-03, 8.92134540e-07, 0.00000000e+00,
        7.03998235e-03, 2.23738421e-02, 1.48046753e-02, 5.76036404e-03,
        4.46067270e-07, 0.00000000e+00, 3.64184188e-03, 1.48781277e-02,
        1.39638385e-02, 4.89246582e-03, 8.92134540e-07, 0.00000000e+00,
        1.48689090e-07, 1.48689090e-07, 0.00000000e+00, 0.00000000e+00,
        2.23033635e-06, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.02754569e-02, 1.69795506e-02, 2.45973388e-02, 1.40139467e-02,
        8.92134540e-07, 0.00000000e+00, 2.13263275e-02, 1.52269523e-02,
        3.69425479e-02, 1.46698143e-02, 1.04082363e-06, 0.00000000e+00,
        2.47568822e-02, 5.26192847e-02, 3.72034972e-02, 1.68320511e-02,
        1.48689090e-06, 0.00000000e+00, 3.50638612e-03, 1.124015